---
<br/>
<div align="center">

# Connect-4 [Artificial Intelligence Project]
</div>

---

<div align="center">

## Search Problem 
</div>

A Search Problem is composed by multiple factors:
- States (Any possible configuration of the Board)
- Initial and Goal States
- Action Space (actions and their effects on the environment)
- Action Cost

---

In [1]:
import numpy as np
import pygame


pygame 2.5.2 (SDL 2.28.2, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Connect4:

    # Defining Board's Size
    NROWS = 6
    NCOLS = 7
    
    def __init__(self, Board_Configuration=None):
        # Matrix to Store the Board's Values
        self.board = np.zeros(shape=(self.NROWS, self.NCOLS), dtype=np.int8) if Board_Configuration is None else Board_Configuration

        # Variable to store the Winner (0 - Tie || 1 - PLayer 1 || 2 - Player 2 / AI)
        self.winner = 0

    def valid_move(self, ncol):
        # Checking if a move can be made (which is basically checking if the collumn is already full)
        return (ncol >= 0 and ncol < self.NCOLS) and (self.board[0][ncol] == 0)

    def move(self, ncol, player):
        if (self.valid_move(ncol)):
            # Looping through the column (Bottom -> Up) where we want to insert a piece
            for (nrow, item) in enumerate(self.board[:, ncol][::-1]):
                if (item == 0):
                    self.board[self.NROWS - nrow - 1][ncol] = player
                    return True
        return False

    def is_full(self):
        # Checks the top Column in order to see if it's Full
        for top_item in self.board[0]:
            if top_item == 0:
                return False
        return True


    def is_over(self):
        # To Implement (Check for any 4 piece combination)
        pass

    def run(self):
        # To Implement (Create Main Loop)
        pass
    
    def __str__(self):
        CODE = {0:'-', 1:'X', 2:'O'} # Maybe helpfull to convert the board into the same style as in the Assignment 1 Paper
        return str(self.board)

    def __hash__(self):
        return hash(str(self.board))

    def __eq__(self, other:object):
        if (not isinstance(other, Connect4)):
            return False
        return hash(self) == hash(other)

In [3]:
game = Connect4()
print(game)

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


In [4]:
game.move(0, 1)
game.move(2, 1)
game.move(0, 1)
print(game.board)

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0]]


In [6]:
game.is_full()

False

---
Graphical User Interface

---